In [1]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json 
from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
import torch.optim as optim

from loguru import logger

from utils import train_validate_model, modify_model_output, test_model

import time
import wandb

image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_20_percent"
train_dir = image_path / 'train' 
test_dir = image_path / 'test'
valid_dir = image_path / 'valid'

device ='cuda' if torch.cuda.is_available() else 'cpu'

2024-06-23 18:24:52.010 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Git\hamburger-hotdog-pizza-classifier


In [2]:
wandb.init(project='pizza_hamburger_hotdog_20_percent')

wandb: Currently logged in as: dtiourine. Use `wandb login --relogin` to force relogin


In [3]:
config_path = 'params.json'

with open(config_path, 'r') as config_file:
   config = json.load(config_file)
#print(json.dumps(config, indent=2))

In [4]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [5]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Start with Pre-Trained Models

Let's choose some popular CNN architectures as a starting point

First, we need to load these pretrained models

In [6]:
num_classes = 3
alexnet = modify_model_output('alexnet', num_classes, device)
vgg16 = modify_model_output('vgg16', num_classes, device)
resnet50 = modify_model_output('resnet50', num_classes, device)

## 1. Trying ResNet50

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.3)
model_name = "ResNet50"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=10, model=resnet50, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device, model_save_path="models/best_resnet50_model.pth")

Overall Training Progress: 100%|██████████| 10/10 [00:17<00:00,  1.79s/it, Best Val Accuracy=59.44%, Current Train Accuracy=55.93%, Current Val Accuracy=59.44%]


## 2. Trying VGG16

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.3)
model_name = "VGG16"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=20, model=vgg16, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device, model_save_path="models/best_vgg16_model.pth")

Overall Training Progress: 100%|██████████| 20/20 [00:49<00:00,  2.45s/it, Best Val Accuracy=87.78%, Current Train Accuracy=91.85%, Current Val Accuracy=87.22%]


## 3. Trying AlexNet

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.3)
model_name = "AlexNet"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=10, model=alexnet, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device, model_save_path="models/best_alexnet_model.pth")

Overall Training Progress: 100%|██████████| 10/10 [00:17<00:00,  1.75s/it, Best Val Accuracy=78.89%, Current Train Accuracy=84.81%, Current Val Accuracy=78.33%]


Since VGG16 was able to achieve nearly 88% accuracy on the validation set, let's start off with VGG16 as our base model to improve upon.